In [1]:
%pip install llama-index
%pip install llama-index-core
%pip install llama-index-llms-anthropic llama-index-multi-modal-llms-anthropic
%pip install llama-index-embeddings-huggingface
%pip install llama-parse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfull

In [10]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"

# Using Anthropic API for embeddings/LLMs
os.environ["ANTHROPIC_API_KEY"] = "sk-"

In [11]:
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(model="claude-3-opus-20240229", temperature=0.0)

In [12]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from llama_parse import LlamaParse

parser = LlamaParse(verbose=True)
json_objs = parser.get_json_result("/content/your.pdf")
json_list = json_objs[0]["pages"]

c585aff3-fdb9-45b8-9f2b-7229a77f4820 John Thurs {'page': 1, 'text': "                                                                                                                                                                      John Thurston\n                                                                                                                                                           Arkansas Secretary of StateBusiness and Commerclal Sarvlcas Divis on\n                                                                                                                                                     Financing Statement            Initial\n UCC FINANCING STATEMENT                                                                                                               Date Filed;11/20/2023 9.37 AM                         Page(s):1\n  FOLLOW INSTRUCTIONS                                                                                                                   

In [ ]:
from llama_index.core.schema import TextNode
from typing import List


def get_text_nodes(json_list: List[dict]):
    text_nodes = []
    for idx, page in enumerate(json_list):
        text_node = TextNode(text=page["text"], metadata={"page": page["page"]})
        text_nodes.append(text_node)
    return text_nodes

In [ ]:
text_nodes = get_text_nodes(json_list)

In [ ]:
# call get_images on parser, convert to ImageDocuments
!mkdir llama2_images

from llama_index.core.schema import ImageDocument
from llama_index.multi_modal_llms.anthropic import AnthropicMultiModal


def get_image_text_nodes(json_objs: List[dict]):
    """Extract out text from images using a multimodal model."""
    anthropic_mm_llm = AnthropicMultiModal(max_tokens=300)
    image_dicts = parser.get_images(json_objs, download_path="llama2_images")
    image_documents = []
    img_text_nodes = []
    for image_dict in image_dicts:
        image_doc = ImageDocument(image_path=image_dict["path"])
        response = anthropic_mm_llm.complete(
            prompt="Describe the images as alt text",
            image_documents=[image_doc],
        )
        text_node = TextNode(text=str(response), metadata={"path": image_dict["path"]})
        img_text_nodes.append(text_node)
    return img_text_nodes

In [ ]:
image_text_nodes = get_image_text_nodes(json_objs)

In [ ]:
image_text_nodes[0].get_content()